In [ ]:
# 基础库导入
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, PromptTuningConfig, TaskType
from datasets import load_dataset
import torch

# 加载数据集
dataset = load_dataset('json', data_files={
    'train': '../data/CAIL/CAIL2018_ALL_DATA/final_all_data/exercise_contest/data_train.json',
    'validation': '../data/CAIL/CAIL2018_ALL_DATA/final_all_data/exercise_contest/data_valid.json',
    'test': '../data/CAIL/CAIL2018_ALL_DATA/final_all_data/exercise_contest/data_test.json'
})

# 加载模型与Tokenizer
model_name = "THUDM/chatglm-6b"
model = AutoModel.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    device_map='auto',
    offload_folder="offload_folder"  # 你后来补充的参数
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Prompt Tuning 配置
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init='TEXT',
    num_virtual_tokens=20,
    tokenizer_name_or_path=model_name
)

# 注入Prompt Tuning参数
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 数据预处理与Tokenization
def preprocess(examples):
    inputs = tokenizer(examples["input"], max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(examples["output"], max_length=512, truncation=True, padding='max_length')
    inputs['labels'] = labels['input_ids']
    return inputs

tokenized_datasets = dataset.map(preprocess, batched=True, remove_columns=dataset['train'].column_names)

# 训练参数配置
training_args = TrainingArguments(
    output_dir="./results_ptuning",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    logging_steps=50,
    save_strategy="epoch"
)

# 初始化 Trainer 并开始训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

trainer.train()
